In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os


import matplotlib.pyplot as plt
import seaborn as sns


DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SEED = 42


def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
#### 노트북 셀 가로 넓게 보기
from IPython.core.display import display, HTML

notebook_config = ""
notebook_config += "<style>"
notebook_config += "  .container { width:90% !important; }"
notebook_config += "</style>"
HTML(notebook_config)

In [ ]:
# SEED 및 프로그램 버전 설정
IS_COLAB = True
PROGRAM_VERSION = "13"

In [ ]:
# if IS_COLAB:
#     from google.colab import drive
#     drive.mount('/content/drive')

In [ ]:
# !pip install bayesian-optimization


In [ ]:
# !pip install scikit-optimize


In [ ]:
!pip install CatBoost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
# import pandas as pd
# import numpy as np
# import random
# import os
# import matplotlib.pyplot as plt
# import warnings; warnings.filterwarnings("ignore")
# import shutil
# import time
# from scipy.stats import gmean
# from itertools import combinations
# from tqdm import tqdm
# tqdm.pandas()

# import tensorflow as tf
# from tensorflow import keras


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Dense
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.layers import TimeDistributed, LayerNormalization
# from tensorflow.keras import layers
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.models import Model

# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# from sklearn.feature_selection import SelectKBest
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import PowerTransformer
# from sklearn.preprocessing import QuantileTransformer

# # Modeling
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import Ridge
# from sklearn.linear_model import ElasticNet
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import ARDRegression
# from sklearn.linear_model import BayesianRidge
# from sklearn.linear_model import HuberRegressor
# from sklearn.linear_model import RANSACRegressor
# from sklearn.linear_model import TheilSenRegressor
# from sklearn.linear_model import PassiveAggressiveRegressor


# from sklearn.ensemble import VotingRegressor
# from sklearn.ensemble import StackingRegressor

# # # AdaBoost
# from sklearn.ensemble import AdaBoostRegressor

# # XGBoost
# from xgboost import plot_importance
# from xgboost import XGBRegressor

# # lightgbm
# from lightgbm import LGBMRegressor

# # # catboost
# from catboost import CatBoostRegressor

# # metric
# from sklearn.metrics import mean_absolute_error

# from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
# from tensorflow.keras.utils import to_categorical

# from sklearn.model_selection import RandomizedSearchCV
# from bayes_opt import BayesianOptimization
# from skopt.space import Real, Categorical, Integer
# from skopt import BayesSearchCV


# import keras

# print(tf.__version__)
# print(keras.__version__)


In [ ]:
# !pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 41.2 MB/s eta 0:00:00


In [ ]:
# !pip install mordred


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.9 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=fe114b6cba0e301cf1607af2b8a4389bcc5b58dc77384a3ced7b446a3a247527
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
# from rdkit import Chem
# from rdkit.Chem import AllChem
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw
# from mordred import Calculator, descriptors

# IPythonConsole.drawOptions.addAtomIndices = True
# IPythonConsole.ipython_useSVG = True
# # IPythonConsole.molSize = 500, 500

In [ ]:
# 일단 colab 에서 실행하는 것으로 설정
DATA_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SUBMISSION_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
CLEANED_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
MODEL_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'

In [ ]:
# train_original = pd.read_csv(f'{DATA_DIR}train.csv')
# # dev = pd.read_csv(f'{DATA_DIR}dev.csv')
# test = pd.read_csv(f'{DATA_DIR}test.csv')
# submission = pd.read_csv(f'{DATA_DIR}sample_submission.csv')

# train = train_original # pd.concat([train_original, dev], axis=0)

# train_original.shape, train.shape, test.shape

In [ ]:
# calc = Calculator(descriptors, ignore_3D=False)
# len(calc.descriptors)

1826

In [ ]:
# x_train = calc.pandas([Chem.MolFromSmiles(x) for x in train.SMILES])
# x_train

In [ ]:
# train_complete = pd.concat([train, x_train], axis=1)
# train_complete

In [ ]:
# x_test = calc.pandas([Chem.MolFromSmiles(x) for x in test.SMILES])
# x_test

In [ ]:
test_complete = pd.concat([test, x_test], axis=1)
test_complete

In [ ]:
# train_complete.to_csv(f"{CLEANED_DIR}train_mordred_1835.csv", index=False)
# test_complete.to_csv(f"{CLEANED_DIR}test_mordred_1835.csv", index=False)

### 데이터 로드

In [ ]:
train = pd.read_csv(f'{CLEANED_DIR}train_mordred_1835.csv')
test = pd.read_csv(f'{CLEANED_DIR}test_mordred_1835.csv')


<ipython-input-9-934881b7f1a5>:1: DtypeWarning: Columns (151,152,153,160,161,162,169,170,171,178,179,180,187,188,189,196,197,198,205,206,207,214,215,216,223,224,225,232,233,234,241,242,243,244,245,246,247,248,249,250,251,252,352,353,354,355,356,357,358,359,360,367,368,369,376,377,378,385,386,387,394,395,396,403,404,405,412,413,414,421,422,423,430,431,432,439,440,441,448,449,450,457,458,459,460,461,462,463,464,465,466,467,473,474,475,481,482,483,489,490,491,497,498,499,505,506,507,513,514,515,521,522,523,529,530,531,537,538,539,545,546,547,553,554,555,556,557,558,559,560,561,562,563,569,570,571,577,578,579,585,586,587,593,594,595,601,602,603,609,610,611,617,618,619,625,626,627,633,634,635,641,642,643,649,650,651,652,653,826,827,1584,1586,1806) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(f'{CLEANED_DIR}train_mordred_1835.csv')


### -> 이상치 제거 (총 18개)

In [ ]:
# # 1. 극단치만 판단해서 18개 제거
# outliers = train[(train['MLM'] > 100.0) | (train['HLM'] > 100.0) | (train['AlogP'] < -3) | (train['Molecular_Weight'] > 800)
#      | (train['Num_H_Acceptors'] > 14) | (train['Num_H_Donors'] > 9) | (train['Num_RotatableBonds'] > 20)
#      | (train['LogD'] < -4) | (train['Molecular_PolarSurfaceArea'] > 250)]


In [ ]:
# outliers.index

Int64Index([ 179,  662,  834,  983, 1092, 1172, 1239, 1584, 2159, 2258, 2367,
            2410, 2586, 2711, 2948, 3157, 3247, 3403],
           dtype='int64')

In [ ]:
# train = train.drop(index=outliers.index, axis=0)

In [ ]:
# train.shape # 3498 -> 3480

(3480, 1837)

In [ ]:
duplicates = train[train.duplicated(subset=['SMILES'], keep=False)]

duplicates.sort_values(by='SMILES')

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
2276,TRAIN_2276,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,1.535,31.453,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
451,TRAIN_0451,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.310,24.670,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2891,TRAIN_2891,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,55.950,69.950,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
543,TRAIN_0543,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,68.485,85.872,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
837,TRAIN_0837,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,63.522,62.488,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
366,TRAIN_0366,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,73.740,66.850,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
1085,TRAIN_1085,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,23.630,71.470,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2848,TRAIN_2848,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,40.657,99.900,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2096,TRAIN_2096,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,1.880,61.280,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
1666,TRAIN_1666,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,5.494,25.397,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000


In [ ]:
processed_duplicate = duplicates.groupby('SMILES')['MLM','HLM'].mean().reset_index()

<ipython-input-11-a090693ccaae>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  processed_duplicate = duplicates.groupby('SMILES')['MLM','HLM'].mean().reset_index()


In [ ]:
tmp = pd.merge(processed_duplicate,duplicates,how='left',on='SMILES')

### -> 중복치 제거 (총 26개)

In [ ]:
tmp = tmp.iloc[0::2]  # 첫 번째 행부터 시작하여 2씩 증가하여 행 선택
tmp.drop(['MLM_y','HLM_y'],axis =1,inplace=True)

<ipython-input-13-d20f759e9146>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.drop(['MLM_y','HLM_y'],axis =1,inplace=True)


In [ ]:
# tmp.columns = ['SMILES', 'MLM', 'HLM','id',  'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']

In [ ]:
# processed_duplicate = tmp[['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']]

In [ ]:
train = train.drop_duplicates(subset=['SMILES'], keep=False) #기존 train에서 중복치들을 우선 모두 제거
train.shape

(3444, 1837)

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [ ]:
tmp.shape

(27, 1837)

In [ ]:
tmp.rename(columns={'HLM_x': 'HLM', 'MLM_x': 'MLM'}, inplace=True)

In [ ]:
train = pd.concat([train, tmp], axis=0, ignore_index=True)
train.shape

(3471, 1837)

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.126,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3467,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.732,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3468,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.840,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222
3469,TRAIN_2159,N#Cc1nccnc1OC1CCN(C(=O)N2CCNC2=O)C1,56.6085,49.519,-0.533,302.289,6,1,2,-0.533,...,9.881191,71.515477,302.112738,8.392021,1133,31,114.0,134.0,6.777778,4.944444


# Mordred Columns확인

-  Object Columns (총 575개)
- train, test 하나라도

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

target_1 = train['MLM']
target_2 = train['HLM']

In [ ]:
train.shape, test.shape

((3471, 1837), (483, 1835))

In [ ]:
test.columns

Index(['id', 'SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'ABC',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1835)

In [ ]:
train.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1837)

In [ ]:
object_columns_train = []

for column in train_prop.columns:
    dtype = train_prop[column].dtype
    # print(f"Column: {column}, Data Type: {dtype}")
    if dtype == object :
        object_columns_train.append(column)


In [ ]:
object_columns_test = []

for column in test_prop.columns:
    dtype = test_prop[column].dtype
    # print(f"Column: {column}, Data Type: {dtype}")
    if dtype == object :
        object_columns_test.append(column)


In [ ]:
object_columns = object_columns_train + object_columns_test

object_columns = set(object_columns)
object_columns = list(object_columns)

len(object_columns_train), len(object_columns_test), len(object_columns)

(575, 403, 575)

In [ ]:
row_info = train_prop.iloc[0]
row_info

AlogP                    3.259
Molecular_Weight       400.495
Num_H_Acceptors              5
Num_H_Donors                 2
Num_RotatableBonds           8
                        ...   
WPol                        40
Zagreb1                  142.0
Zagreb2                  165.0
mZagreb1                   9.5
mZagreb2              6.361111
Name: 0, Length: 1833, dtype: object

- pure object columns(str) -> 삭제 (319개)
    - object칼럼 중 합계 0 인 칼럼
    - 행별로 동일해서 의미없는 object column으로 판단함

In [ ]:
# # 열(column_name)의 숫자 데이터만 추출하여 합계 계산
# numeric_values = pd.to_numeric(train_prop[object_columns[0]], errors='coerce')
# column_sum = numeric_values.sum()
# print(i, column_sum)

In [ ]:
pure_object_cols_train = []

for i in range(len(object_columns)):
    numeric_values = pd.to_numeric(train_prop[object_columns[i]], errors='coerce')
    column_sum = numeric_values.sum()
    # print('column',i,':', column_sum)
    if column_sum == 0.0:
        pure_object_cols_train.append(object_columns[i])


In [ ]:
pure_object_cols_test = []

for i in range(len(object_columns)):
    numeric_values = pd.to_numeric(test_prop[object_columns[i]], errors='coerce')
    column_sum = numeric_values.sum()
    # print('column',i,':', column_sum)
    if column_sum == 0.0:
        pure_object_cols_test.append(object_columns[i])


In [ ]:
pure_object_cols = pure_object_cols_train + pure_object_cols_test

pure_object_cols = set(pure_object_cols)
pure_object_cols = list(pure_object_cols)


len(pure_object_cols_train), len(pure_object_cols_test), len(pure_object_cols)

(313, 317, 319)

In [ ]:
len(object_columns), len(pure_object_cols)

# 575개 중에 pure object cols 313개는 object(catboost로) / 나머지는 머신러닝으로 -> 앙상블?

(575, 319)

In [ ]:
train[pure_object_cols]


,MAXsSnH3,MAXsPH2,Mor21m,FNSA5,WPSA5,MAXsSH,Mor28v,PPSA2,GRAVp,MINsssGeH,...,Mor08p,Mor23v,Mor11,MAXsssdAs,MAXssssN,Mor28p,MINsssssP,Mor05m,MINsssPbH,MINsPbH3
0,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
1,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
2,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
3,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
4,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mord

In [ ]:
train[pure_object_cols]

,MAXsSnH3,MAXsPH2,Mor21m,FNSA5,WPSA5,MAXsSH,Mor28v,PPSA2,GRAVp,MINsssGeH,...,Mor08p,Mor23v,Mor11,MAXsssdAs,MAXssssN,Mor28p,MINsssssP,Mor05m,MINsssPbH,MINsPbH3
0,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
1,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
2,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
3,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),max() arg is an empty sequence (MAXsSH),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (GRAV/mordred._graph_mat...,min() arg is an empty sequence (MINsssGeH),...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssdAs),max() arg is an empty sequence (MAXssssN),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssssP),missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssPbH),min() arg is an empty sequence (MINsPbH3)
4,max() arg is an empty sequence (MAXsSnH3),max() arg is an empty sequence (MAXsPH2),missing 3D coordinate (GeomDiameter/mord

- float object columns -> Catboost (256개)

In [ ]:
float_object_cols = [x for x in object_columns if x not in pure_object_cols]
float_object_cols

['MATS6c',
 'AATS8are',
 'MAXaaaC',
 'AATSC6dv',
 'AATSC5c',
 'AATSC2c',
 'GATS5c',
 'MATS8pe',
 'MATS6dv',
 'AATS8dv',
 'MINdssC',
 'MATS7dv',
 'MDEC-24',
 'MINssS',
 'AATS6v',
 'AATSC6c',
 'MDEC-13',
 'AATSC8dv',
 'MAXaaN',
 'MINaaO',
 'MAXdO',
 'AATSC6s',
 'MAXaaNH',
 'AATSC6v',
 'GATS3c',
 'GATS8c',
 'MATS6m',
 'MAXaaCH',
 'MAXsF',
 'ATSC2c',
 'AATS7m',
 'AATSC7i',
 'GATS8p',
 'AATS6Z',
 'AATSC6are',
 'MATS7p',
 'AATSC8i',
 'GATS8m',
 'MINsssCH',
 'GATS2c',
 'MINdNH',
 'MINsBr',
 'MAXaaS',
 'MATS7are',
 'GATS7i',
 'MDEN-12',
 'MINaasN',
 'GATS6dv',
 'MDEC-23',
 'MINdssS',
 'MINddsN',
 'ATSC6c',
 'Vabc',
 'MAXsssCH',
 'MATS8d',
 'MDEN-22',
 'MAXssCH2',
 'MINdCH2',
 'RPCG',
 'ATSC5c',
 'MATS2c',
 'AATSC8p',
 'MDEN-13',
 'AATSC1c',
 'AATS6d',
 'MAXsNH2',
 'GATS6i',
 'AATSC7dv',
 'MINsNH2',
 'GATS7are',
 'MINtN',
 'GATS6se',
 'AATSC7are',
 'AATSC6Z',
 'MATS8i',
 'GATS4c',
 'MINdO',
 'AATSC6pe',
 'MAXddsN',
 'AATS7pe',
 'AATSC7d',
 'MATS7se',
 'MAXdNH',
 'AATSC6m',
 'GATS8v',
 'MDEC-22'

In [ ]:
len(float_object_cols)

256

In [ ]:
train[float_object_cols]

,MATS6c,AATS8are,MAXaaaC,AATSC6dv,AATSC5c,AATSC2c,GATS5c,MATS8pe,MATS6dv,AATS8dv,...,MINaaS,MAXdssS,AATSC7c,AATSC6se,AATS7d,MDEC-11,MATS6v,MINsCH3,MATS8m,GATS8se
0,-0.034854,5.940759615384615,max() arg is an empty sequence (MAXaaaC),-0.45428,0.000283,-0.002626,1.111785,0.11351599436193428,-0.110333,2.900641025641026,...,1.5631235656838525,max() arg is an empty sequence (MAXdssS),-0.000848,-0.0066,2.411215,0.5665259691646413,-0.022033,1.9229353985270639,0.08816542374445727,0.8519407266988336
1,-0.003946,5.76492,max() arg is an empty sequence (MAXaaaC),0.0607,-0.000291,0.000619,0.890061,-0.032281010458276665,0.016939,1.4,...,1.73338813303099,max() arg is an empty sequence (MAXdssS),0.0005,-0.001192,2.376471,0.49397531700626224,0.14366,1.9712226001511715,-0.14280236626985532,0.8833231118180888
2,0.022825,6.068255555555556,1.1192051156882508,-0.528667,0.000299,-0.002289,0.91353,-0.1527379967326501,-0.136218,2.3703703703703702,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),-0.000533,-0.004472,2.418605,float division by zero (MDEC-11),-0.076475,2.2292460165629358,-0.13142734431168296,1.2494283405961528
3,0.018565,6.0857943262411345,max() arg is an empty sequence (MAXaaaC),0.330945,-0.00124,-0.0006,1.239,-0.1786562941869179,0.090445,3.00709219858156,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),0.001518,0.004183,2.517241,0.37117320236996454,-0.035476,1.7527397714518926,-0.002535843563919458,1.7574443294470312
4,0.042476,5.699627118644067,max() arg is an empty sequence (MAXaaaC),-0.889149,0.002423,-0.001388,0.901147,0.04736676437451053,-0.222459,1.5932203389830508,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),-0.001477,-0.008586,2.197531,0.14285714285714285,-0.218617,1.9678829680020156,-0.14042559082908032,0.5562006677345391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,0.06083689928513764,5.872409,max() arg is an empty sequence (MAXaaaC),-0.369649,-0.0001459584417922954,-0.002530326616976849,1.1509274094425375,0.094792,-0.09766,2.889984,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),-0.0008675842556499156,0.00152,2.671717,0.09090909090909091,-0.063538,1.8070926141815091,0.149063,0.779814
3467,0.104772,6.023862345679012,0.9407641513395483,-0.066641,-0.001497,0.003046,1.240231,-0.0051280672356559505,-0.015946,3.6831275720164616,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),-0.001078,0.010955,2.962733,0.388067491097121,-0.081284,1.0529719014684744,0.07052283005424888,0.972267074423757
3468,-0.010238,6.412925925925926,max() arg is an empty sequence (MAXaaaC),-0.020833,0.001828,-0.00163,0.675558,0.01879636927773421,-0.004701,4.54320987654321,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),-0.002282,0.000148,2.522523,float division by zero (MDEC-11),0.001611,min() arg is an empty sequence (MINsCH3),-0.13075740193989047,1.003050500058162
3469,-0.142797,6.343617,max() arg is an empty sequence (MAXaaaC),-0.597737,0.001998,0.005141,0.887103,0.175862,-0.126167,6.333333,...,min() arg is an empty sequence (MINaaS),max() arg is an empty sequence (MAXdssS),0.004419,-0.01277,2.27027,float division by zero (MDEC-11),-0.299173,min() arg is an empty sequence (MINsCH3),0.460391,0.953556


In [ ]:
row_types = []

#iterrows
for index, row in train_prop.iterrows():
    row_data_types = row.apply(lambda x: type(x))
    row_types.append(row_data_types)

row_types_df = pd.DataFrame(row_types, columns=train_prop.columns)

# row_types_df

In [ ]:
# row_types_df[pure_object_cols]

In [ ]:
# row_types_df[float_object_cols]

In [ ]:
#예시
# pd.DataFrame(train_prop['AATS6dv'])

- float division by zero 다수 포함. 이 부분을 그대로 cat으로 학습시키는게 좋겠다고 판단함

In [ ]:
# train_prop[float_object_cols].astype(float)
# float division by zero

# train_prop[float_object_cols] = train_prop[float_object_cols].apply(lambda x: 0 if isinstance(x, str) else x)

train_prop[float_object_cols]

,MINdS,GATS8Z,AATSC2c,AATS6Z,AATSC8pe,AATS6i,MATS8i,MINaasC,GATS3c,AATSC5c,...,ATSC0c,AATS8v,AATS7d,AATS6pe,MINsOH,GATS7Z,MDEC-24,MINsF,AATS6se,MATS6s
0,min() arg is an empty sequence (MINdS),0.8788098560517039,-0.002626,14.512,0.013116789940828356,163.994562,0.09146741812001007,0.4133529570510992,1.228269,0.000283,...,1.156764,141.25475137438323,2.411215,6.099424,min() arg is an empty sequence (MINsOH),0.82095,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.640306,-0.138399
1,min() arg is an empty sequence (MINdS),0.8995260663507108,0.000619,15.324074,-0.0027334833333333354,161.517345,-0.09828343058060138,0.8942039871504159,1.372832,-0.000291,...,0.546184,128.35945225100608,2.376471,5.809655,min() arg is an empty sequence (MINsOH),1.383538,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.375657,-0.045418
2,min() arg is an empty sequence (MINdS),1.1472914085661632,-0.002289,12.533981,-0.01398972833630774,167.546644,0.157932834834016,0.9782856198034771,1.1944,0.000299,...,0.386576,143.29793565140298,2.418605,5.990433,min() arg is an empty sequence (MINsOH),1.094076,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.491145,-0.147724
3,min() arg is an empty sequence (MINdS),1.151027286015917,-0.0006,14.424731,-0.016699946819682905,164.481915,0.18202592172169685,0.28521064289913545,1.370712,-0.00124,...,0.5426,164.31489701866522,2.517241,5.954312,min() arg is an empty sequence (MINsOH),1.099364,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.476534,0.088347
4,min() arg is an empty sequence (MINdS),1.1202433200184025,-0.001388,14.180723,0.00500029294831561,155.229558,-0.2347065002903171,0.6403297430083146,1.353545,0.002423,...,0.646734,124.78549465606643,2.197531,6.049133,min() arg is an empty sequence (MINsOH),1.048877,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.525117,-0.124405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,min() arg is an empty sequence (MINdS),1.2451907037151777,-0.001579,16.1375,-0.03745383141762453,164.796348,0.18173180602575226,0.6033961115310325,0.904475,-0.001698,...,0.485966,200.60292202728104,3.380952,6.090751,min() arg is an empty sequence (MINsOH),1.002332,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.538888,0.162221
3450,min() arg is an empty sequence (MINdS),0.625184,-0.002530326616976849,18.491979,0.010368,158.87219,0.068619,0.29657090644944994,0.9975419161586702,-0.0001459584417922954,...,1.0737189041606328,157.475974,2.671717,6.240779,min() arg is an empty sequence (MINsOH),0.936707,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.708733,-0.019877
3451,min() arg is an empty sequence (MINdS),0.8826295330532619,0.003046,19.376623,-0.000575537265660806,156.481164,-0.07305405146309424,-0.20248726008105478,1.145175,-0.001497,...,0.85529,172.50521019668182,2.962733,6.274321,min() arg is an empty sequence (MINsOH),1.009788,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.67878,0.1916
3452,min() arg is an empty sequence (MINdS),1.1022927689594355,-0.00163,13.138889,0.002618518518518548,166.9343,0.04582660667269332,0.1396013721938485,1.169655,0.001828,...,1.207438,177.24611364442714,2.522523,6.091397,min() arg is an empty sequence (MINsOH),1.091653,float division by zero (MDEC-24),min() arg is an empty sequence (MINsF),7.639349,-0.035673


- SMILES, ID -> 보류

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


- Float Columns (총 1258개)

In [ ]:
float_columns = [x for x in train_prop.columns if x not in object_columns]

In [ ]:
# float_columns

In [ ]:
len(float_columns)

1258

# 1) CatBoost - object columns
- https://catboost.ai/en/docs/concepts/python-reference_catboostregressor

In [ ]:
# class CatBoostRegressor(iterations=None,
#                         learning_rate=None,
#                         depth=None,
#                         l2_leaf_reg=None,
#                         model_size_reg=None,
#                         rsm=None,
#                         loss_function='RMSE',
#                         border_count=None,
#                         feature_border_type=None,
#                         per_float_feature_quantization=None,
#                         input_borders=None,
#                         output_borders=None,
#                         fold_permutation_block=None,
#                         od_pval=None,
#                         od_wait=None,
#                         od_type=None,
#                         nan_mode=None,
#                         counter_calc_method=None,
#                         leaf_estimation_iterations=None,
#                         leaf_estimation_method=None,
#                         thread_count=None,
#                         random_seed=None,
#                         use_best_model=None,
#                         best_model_min_trees=None,
#                         verbose=None,
#                         silent=None,
#                         logging_level=None,
#                         metric_period=None,
#                         ctr_leaf_count_limit=None,
#                         store_all_simple_ctr=None,
#                         max_ctr_complexity=None,
#                         has_time=None,
#                         allow_const_label=None,
#                         one_hot_max_size=None,
#                         random_strength=None,
#                         name=None,
#                         ignored_features=None,
#                         train_dir=None,
#                         custom_metric=None,
#                         eval_metric=None,
#                         bagging_temperature=None,
#                         save_snapshot=None,
#                         snapshot_file=None,
#                         snapshot_interval=None,
#                         fold_len_multiplier=None,
#                         used_ram_limit=None,
#                         gpu_ram_part=None,
#                         pinned_memory_size=None,
#                         allow_writing_files=None,
#                         final_ctr_computation_mode=None,
#                         approx_on_full_history=None,
#                         boosting_type=None,
#                         simple_ctr=None,
#                         combinations_ctr=None,
#                         per_feature_ctr=None,
#                         ctr_target_border_count=None,
#                         task_type=None,
#                         device_config=None,
#                         devices=None,
#                         bootstrap_type=None,
#                         subsample=None,
#                         sampling_unit=None,
#                         dev_score_calc_obj_block_size=None,
#                         max_depth=None,
#                         n_estimators=None,
#                         num_boost_round=None,
#                         num_trees=None,
#                         colsample_bylevel=None,
#                         random_state=None,
#                         reg_lambda=None,
#                         objective=None,
#                         eta=None,
#                         max_bin=None,
#                         gpu_cat_features_storage=None,
#                         data_partition=None,
#                         metadata=None,
#                         early_stopping_rounds=None,
#                         cat_features=None,
#                         grow_policy=None,
#                         min_data_in_leaf=None,
#                         min_child_samples=None,
#                         max_leaves=None,
#                         num_leaves=None,
#                         score_function=None,
#                         leaf_estimation_backtracking=None,
#                         ctr_history_unit=None,
#                         monotone_constraints=None,
#                         feature_weights=None,
#                         penalties_coefficient=None,
#                         first_feature_use_penalties=None,
#                         model_shrink_rate=None,
#                         model_shrink_mode=None,
#                         langevin=None,
#                         diffusion_temperature=None,
#                         posterior_sampling=None,
#                         boost_from_average=None,
#                         fixed_binary_splits=None) :


### 학습 및 검증

In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,3.259,400.495,5,2,8,3.259,117.37,21.379612,17.449011,0,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,2.169,301.407,2,1,2,2.172,73.47,16.539255,14.049653,0,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,1.593,297.358,5,0,3,1.585,62.45,17.475469,13.660693,2,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,4.771,494.652,6,0,5,3.475,92.60,27.857311,20.034364,0,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,2.335,268.310,3,0,1,2.337,42.43,15.722758,12.817176,0,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5,0,7,4.009,65.98,30.031303,22.567906,0,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3467,4.304,459.520,6,1,4,4.304,113.53,26.501865,20.702897,0,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3468,-1.133,385.417,7,0,4,-1.133,108.65,21.936889,17.403250,0,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222
3469,-0.533,302.289,6,1,2,-0.533,111.44,17.027582,14.772010,0,...,9.881191,71.515477,302.112738,8.392021,1133,31,114.0,134.0,6.777778,4.944444


In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,3.259,400.495,5,2,8,3.259,117.37,21.379612,17.449011,0,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,2.169,301.407,2,1,2,2.172,73.47,16.539255,14.049653,0,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,1.593,297.358,5,0,3,1.585,62.45,17.475469,13.660693,2,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,4.771,494.652,6,0,5,3.475,92.60,27.857311,20.034364,0,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,2.335,268.310,3,0,1,2.337,42.43,15.722758,12.817176,0,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5,0,7,4.009,65.98,30.031303,22.567906,0,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3467,4.304,459.520,6,1,4,4.304,113.53,26.501865,20.702897,0,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3468,-1.133,385.417,7,0,4,-1.133,108.65,21.936889,17.403250,0,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222
3469,-0.533,302.289,6,1,2,-0.533,111.44,17.027582,14.772010,0,...,9.881191,71.515477,302.112738,8.392021,1133,31,114.0,134.0,6.777778,4.944444


In [ ]:
from catboost import CatBoostClassifier


In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [ ]:
train_prop.columns.tolist()

['AlogP',
 'Molecular_Weight',
 'Num_H_Acceptors',
 'Num_H_Donors',
 'Num_RotatableBonds',
 'LogD',
 'Molecular_PolarSurfaceArea',
 'ABC',
 'ABCGG',
 'nAcid',
 'nBase',
 'SpAbs_A',
 'SpMax_A',
 'SpDiam_A',
 'SpAD_A',
 'SpMAD_A',
 'LogEE_A',
 'VE1_A',
 'VE2_A',
 'VE3_A',
 'VR1_A',
 'VR2_A',
 'VR3_A',
 'nAromAtom',
 'nAromBond',
 'nAtom',
 'nHeavyAtom',
 'nSpiro',
 'nBridgehead',
 'nHetero',
 'nH',
 'nB',
 'nC',
 'nN',
 'nO',
 'nS',
 'nP',
 'nF',
 'nCl',
 'nBr',
 'nI',
 'nX',
 'ATS0dv',
 'ATS1dv',
 'ATS2dv',
 'ATS3dv',
 'ATS4dv',
 'ATS5dv',
 'ATS6dv',
 'ATS7dv',
 'ATS8dv',
 'ATS0d',
 'ATS1d',
 'ATS2d',
 'ATS3d',
 'ATS4d',
 'ATS5d',
 'ATS6d',
 'ATS7d',
 'ATS8d',
 'ATS0s',
 'ATS1s',
 'ATS2s',
 'ATS3s',
 'ATS4s',
 'ATS5s',
 'ATS6s',
 'ATS7s',
 'ATS8s',
 'ATS0Z',
 'ATS1Z',
 'ATS2Z',
 'ATS3Z',
 'ATS4Z',
 'ATS5Z',
 'ATS6Z',
 'ATS7Z',
 'ATS8Z',
 'ATS0m',
 'ATS1m',
 'ATS2m',
 'ATS3m',
 'ATS4m',
 'ATS5m',
 'ATS6m',
 'ATS7m',
 'ATS8m',
 'ATS0v',
 'ATS1v',
 'ATS2v',
 'ATS3v',
 'ATS4v',
 'ATS5v',
 '

In [ ]:
train_prop_object = train_prop_object.astype(str)


NameError: ignored

### 1) train prop 전체를 str변환 및 cat feature화

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_object, target_1)):
    # 학습데이터
    x_train = train_prop_object.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_object.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    # model = CatBoostRegressor(random_state=SEED, verbose=0,cat_features=train_prop_object.columns.tolist())
    model = CatBoostRegressor(random_state=SEED, verbose=0,cat_features=train_prop_object.columns.tolist())

    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0, cat_features=train_prop_object.columns.tolist())
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

### 2) train prop전체를 범주화

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

target_1 = train['MLM']
target_2 = train['HLM']

In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')




In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')




In [ ]:
train_prop.isna().sum().sum(), test_prop.isna().sum().sum()

(1304637, 181556)

In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,3.259,400.495,5,2,8,3.259,117.37,21.379612,17.449011,0,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,2.169,301.407,2,1,2,2.172,73.47,16.539255,14.049653,0,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,1.593,297.358,5,0,3,1.585,62.45,17.475469,13.660693,2,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,4.771,494.652,6,0,5,3.475,92.60,27.857311,20.034364,0,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,2.335,268.310,3,0,1,2.337,42.43,15.722758,12.817176,0,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5,0,7,4.009,65.98,30.031303,22.567906,0,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3467,4.304,459.520,6,1,4,4.304,113.53,26.501865,20.702897,0,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3468,-1.133,385.417,7,0,4,-1.133,108.65,21.936889,17.403250,0,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222
3469,-0.533,302.289,6,1,2,-0.533,111.44,17.027582,14.772010,0,...,9.881191,71.515477,302.112738,8.392021,1133,31,114.0,134.0,6.777778,4.944444


In [ ]:
train_prop.shape[0] * train_prop.shape[1]

6331182

In [ ]:
train_prop.fillna(0, inplace=True)
test_prop.fillna(0, inplace=True)

In [ ]:
train_prop.isna().sum().sum(), test_prop.isna().sum().sum()

(0, 0)

#### -> quantile로 범주화하기 ( 1)







In [ ]:
# num_bins = 10  # 원하는 범주 수

# for col in train_prop.columns:
#     bin_edges = pd.qcut(train_prop[col], q=num_bins, labels=False)
#     bin_labels = [str(i) for i in range(num_bins)]
#     train_prop[f'{col}_Category'] = pd.Categorical(bin_edges, categories=range(num_bins), ordered=True)
#     train_prop[f'{col}_Category'] = train_prop[f'{col}_Category'].cat.rename_categories(bin_labels)

# # 결과 출력
# print(train_prop)

import pandas as pd

# 각 열을 0~10 범주로 나누기
num_bins = 10  # 원하는 범주 수

for col in train_prop.columns:
    bin_edges = pd.qcut(train_prop[col], q=num_bins, labels=False, duplicates='drop')
    bin_labels = [str(i) for i in range(num_bins)]
    train_prop[f'{col}_Category'] = pd.Categorical(bin_edges, categories=range(num_bins), ordered=True)
    train_prop[f'{col}_Category'] = train_prop[f'{col}_Category'].cat.rename_categories(bin_labels)

# 결과 출력
print(train_prop)





      AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0     3.259           400.495                5             2   
1     2.169           301.407                2             1   
2     1.593           297.358                5             0   
3     4.771           494.652                6             0   
4     2.335           268.310                3             0   
...     ...               ...              ...           ...   
3466  4.282           553.480                5             0   
3467  4.304           459.520                6             1   
3468 -1.133           385.417                7             0   
3469 -0.533           302.289                6             1   
3470  5.630           466.572                4             0   

      Num_RotatableBonds   LogD  Molecular_PolarSurfaceArea        ABC  \
0                      8  3.259                      117.37  21.379612   
1                      2  2.172                       73.47  16.539255   
2        

In [ ]:
test_prop = test_prop.astype(float)

In [ ]:
test_prop.fillna(0, inplace=True)

In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10_Category,TSRW10_Category,MW_Category,AMW_Category,WPath_Category,WPol_Category,Zagreb1_Category,Zagreb2_Category,mZagreb1_Category,mZagreb2_Category
0,3.259,400.495,5,2,8,3.259,117.37,21.379612,17.449011,0,...,4,7,7,4,7,5,6,5,7,7
1,2.169,301.407,2,1,2,2.172,73.47,16.539255,14.049653,0,...,2,3,2,3,1,3,2,2,3,2
2,1.593,297.358,5,0,3,1.585,62.45,17.475469,13.660693,2,...,4,4,2,2,2,3,3,3,0,3
3,4.771,494.652,6,0,5,3.475,92.60,27.857311,20.034364,0,...,9,9,9,2,9,9,9,9,8,9
4,2.335,268.310,3,0,1,2.337,42.43,15.722758,12.817176,0,...,2,0,0,3,1,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5,0,7,4.009,65.98,30.031303,22.567906,0,...,9,9,9,5,9,9,9,9,9,9
3467,4.304,459.520,6,1,4,4.304,113.53,26.501865,20.702897,0,...,9,9,8,7,8,9,9,9,8,8
3468,-1.133,385.417,7,0,4,-1.133,108.65,21.936889,17.403250,0,...,5,7,6,3,7,5,6,6,4,7
3469,-0.533,302.289,6,1,2,-0.533,111.44,17.027582,14.772010,0,...,1,4,2,6,3,1,2,2,2,3


In [ ]:
test_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,2.641,361.505,4.0,2.0,7.0,2.635,92.76,19.229782,15.428567,0.0,...,9.935713,72.422574,361.193631,6.946031,1607.0,34.0,126.0,144.0,7.527778,5.638889
1,0.585,370.399,5.0,0.0,3.0,0.585,68.31,21.229782,16.881776,0.0,...,10.330421,76.111616,370.152872,7.554140,1853.0,45.0,146.0,175.0,7.840278,5.972222
2,4.276,347.414,4.0,4.0,5.0,4.290,92.86,20.155669,15.481463,0.0,...,10.008163,60.614034,347.174610,7.386694,1905.0,37.0,132.0,150.0,8.388889,5.722222
3,1.795,345.358,5.0,0.0,2.0,1.795,81.21,21.079953,16.076404,0.0,...,10.288444,77.464779,345.133808,8.417898,1673.0,42.0,146.0,177.0,6.000000,5.583333
4,1.219,353.418,4.0,0.0,2.0,0.169,61.15,20.620135,16.065299,0.0,...,10.380312,75.874629,353.185175,7.207861,1656.0,47.0,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,4.207,306.443,2.0,1.0,7.0,4.207,55.13,16.291966,14.728760,0.0,...,9.862509,69.656439,306.230728,5.889052,1138.0,32.0,108.0,125.0,7.756944,5.222222
479,-0.608,335.398,5.0,0.0,1.0,-1.736,70.16,18.777305,15.822051,0.0,...,10.288852,73.647870,335.184506,6.840500,1280.0,42.0,130.0,157.0,9.201389,5.180556
480,1.792,349.383,3.0,1.0,3.0,1.792,69.72,20.648645,16.775304,0.0,...,10.447235,76.277596,349.142641,7.758725,1469.0,48.0,144.0,177.0,8.722222,5.611111
481,0.790,341.132,3.0,2.0,2.0,0.423,69.64,15.343821,13.245018,0.0,...,9.874728,53.565348,339.985883,11.332863,826.0,31.0,102.0,118.0,8.388889,4.361111


In [ ]:
num_bins = 10  # 원하는 범주 수

for col in test_prop.columns:
    bin_edges = pd.qcut(test_prop[col], q=num_bins, labels=False, duplicates='drop')
    bin_labels = [str(i) for i in range(num_bins)]
    test_prop[f'{col}_Category'] = pd.Categorical(bin_edges, categories=range(num_bins), ordered=True)
    test_prop[f'{col}_Category'] = test_prop[f'{col}_Category'].cat.rename_categories(bin_labels)

# 결과 출력
print(test_prop)


     AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0    2.641           361.505              4.0           2.0   
1    0.585           370.399              5.0           0.0   
2    4.276           347.414              4.0           4.0   
3    1.795           345.358              5.0           0.0   
4    1.219           353.418              4.0           0.0   
..     ...               ...              ...           ...   
478  4.207           306.443              2.0           1.0   
479 -0.608           335.398              5.0           0.0   
480  1.792           349.383              3.0           1.0   
481  0.790           341.132              3.0           2.0   
482  2.782           250.380              2.0           0.0   

     Num_RotatableBonds   LogD  Molecular_PolarSurfaceArea        ABC  \
0                   7.0  2.635                       92.76  19.229782   
1                   3.0  0.585                       68.31  21.229782   
2                   5.0 

In [ ]:
train_prop = train_prop.astype(float)

(3454, 1837)

In [ ]:
train_prop.iloc[:,:1833]

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,3.259,400.495,5.0,2.0,8.0,3.259,117.37,21.379612,17.449011,0.0,...,10.081676,78.761075,400.156912,7.695325,2380.0,40.0,142.0,165.0,9.500000,6.361111
1,2.169,301.407,2.0,1.0,2.0,2.172,73.47,16.539255,14.049653,0.0,...,9.907828,69.149596,301.124883,7.528122,870.0,35.0,112.0,132.0,7.138889,4.527778
2,1.593,297.358,5.0,0.0,3.0,1.585,62.45,17.475469,13.660693,2.0,...,10.144510,70.158066,297.170194,7.248054,1028.0,36.0,120.0,145.0,5.277778,4.888889
3,4.771,494.652,6.0,0.0,5.0,3.475,92.60,27.857311,20.034364,0.0,...,10.613467,86.199585,494.246395,7.162991,4170.0,61.0,192.0,231.0,10.784722,7.500000
4,2.335,268.310,3.0,0.0,1.0,2.337,42.43,15.722758,12.817176,0.0,...,9.978363,53.872357,268.121178,7.447810,762.0,32.0,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5.0,0.0,7.0,4.009,65.98,30.031303,22.567906,0.0,...,10.593756,88.164476,552.169496,8.120140,4874.0,63.0,204.0,242.0,12.055556,8.222222
3467,4.304,459.520,6.0,1.0,4.0,4.304,113.53,26.501865,20.702897,0.0,...,10.677177,84.226168,459.136511,8.502528,2888.0,61.0,186.0,227.0,10.895833,6.944444
3468,-1.133,385.417,7.0,0.0,4.0,-1.133,108.65,21.936889,17.403250,0.0,...,10.177818,79.094663,385.175004,7.552451,2262.0,42.0,148.0,175.0,8.000000,6.222222
3469,-0.533,302.289,6.0,1.0,2.0,-0.533,111.44,17.027582,14.772010,0.0,...,9.881191,71.515477,302.112738,8.392021,1133.0,31.0,114.0,134.0,6.777778,4.944444


In [ ]:
train_prop_quantile = train_prop.iloc[:,1833:]
train_prop_quantile

,AlogP_Category,Molecular_Weight_Category,Num_H_Acceptors_Category,Num_H_Donors_Category,Num_RotatableBonds_Category,LogD_Category,Molecular_PolarSurfaceArea_Category,ABC_Category,ABCGG_Category,nAcid_Category,...,SRW10_Category,TSRW10_Category,MW_Category,AMW_Category,WPath_Category,WPol_Category,Zagreb1_Category,Zagreb2_Category,mZagreb1_Category,mZagreb2_Category
0,5.0,7.0,3.0,1.0,5.0,6.0,9.0,6.0,7.0,0.0,...,4.0,7.0,7.0,4.0,7.0,5.0,6.0,5.0,7.0,7.0
1,3.0,2.0,0.0,0.0,0.0,3.0,4.0,2.0,2.0,0.0,...,2.0,3.0,2.0,3.0,1.0,3.0,2.0,2.0,3.0,2.0
2,2.0,2.0,3.0,0.0,1.0,2.0,2.0,3.0,2.0,0.0,...,4.0,4.0,2.0,2.0,2.0,3.0,3.0,3.0,0.0,3.0
3,9.0,9.0,4.0,0.0,3.0,6.0,6.0,9.0,9.0,0.0,...,9.0,9.0,9.0,2.0,9.0,9.0,9.0,9.0,8.0,9.0
4,3.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,8.0,9.0,3.0,0.0,5.0,7.0,3.0,9.0,9.0,0.0,...,9.0,9.0,9.0,5.0,9.0,9.0,9.0,9.0,9.0,9.0
3467,8.0,8.0,4.0,0.0,2.0,8.0,8.0,9.0,9.0,0.0,...,9.0,9.0,8.0,7.0,8.0,9.0,9.0,9.0,8.0,8.0
3468,0.0,6.0,5.0,0.0,2.0,0.0,8.0,7.0,7.0,0.0,...,5.0,7.0,6.0,3.0,7.0,5.0,6.0,6.0,4.0,7.0
3469,0.0,2.0,4.0,0.0,0.0,0.0,8.0,2.0,3.0,0.0,...,1.0,4.0,2.0,6.0,3.0,1.0,2.0,2.0,2.0,3.0


In [ ]:
test_prop_quantile = test_prop.iloc[:,1833:]

In [ ]:
!pip install CatBoost

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_quantile, target_1)):
    # 학습데이터
    x_train = train_prop_quantile.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_quantile.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    # model = CatBoostRegressor(random_state=SEED, verbose=0,cat_features=train_prop_object.columns.tolist())
    model = CatBoostRegressor(random_state=SEED, verbose=0)

    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

0it [00:00, ?it/s]

[30.55807392643382, 31.054386945667193, 30.934236701467476, 31.21567700184533, 31.57118406362588]


31.066711727807938

In [ ]:
test_prop_quantile = test_prop_quantile.astype(float)

In [ ]:
model.fit(train_prop_quantile, target_1)
cat_pred_1_quantile = model.predict(test_prop_quantile)

model.fit(train_prop_quantile, target_2)
cat_pred_2_quantile = model.predict(test_prop_quantile)

In [ ]:
cat_pred_1_quantile

array([19.10345145, 52.22507914, 27.71742171, 40.67011897, 68.7074471 ,
       33.7110421 , 25.43512732, 31.66088617, 26.54855358, 16.71568779,
       42.30173482, 44.35992772, 56.53773903, 17.51750097, 16.97246442,
       58.68238262, 35.18202312, 51.07667108, 53.64753573, 52.59058507,
       16.67036794, 12.09787758, 17.41403791, 18.71023297, 14.87965931,
       23.73308881, 30.71257587, 65.1137756 , 30.75793514, 44.53246163,
       62.34833687, 23.98910326, 14.32684174, 30.08965227, 27.28776069,
       18.64694325, 37.85197356, 47.6806461 , 11.2431655 , 29.45518138,
       25.54601334, 61.6511155 , 22.61789029, 18.27287171, 16.38208738,
       53.56772912,  5.16614957,  0.3375459 , 47.26721742, 21.674221  ,
       70.42114959, 36.63687761, 29.43805288, 31.03957535, 29.23964285,
       14.11486691, 36.03419037, 44.69928474, 11.46901798, 43.90135637,
       24.54477881, 45.14188363, 31.9203268 , 64.83588678, 19.61209507,
       33.86056172, 59.2133151 , 70.60224612, 30.39800576, 39.23

In [ ]:
cat_pred_2_quantile

array([46.07523324, 78.59672655, 55.28717459, 65.56529571, 72.65861763,
       66.23227029, 39.15331485, 43.73514206, 47.3729715 , 37.84333427,
       44.82561341, 78.00529465, 68.67191618, 49.0985311 , 29.21600622,
       87.00652446, 44.10721698, 61.81426449, 71.11788992, 66.52598422,
       35.95321795, 28.37226641, 18.85783816, 31.83009251, 39.20310241,
       52.39746235, 35.69235729, 81.30478391, 44.13001454, 62.91858408,
       65.24895497, 43.57921011, 46.3503308 , 42.63113374, 52.92821187,
       21.26393666, 49.91372895, 72.01934898, 47.00629347, 32.59191741,
       49.12706666, 90.77169591, 37.2504914 , 38.28693663, 24.21502663,
       63.698451  , 41.45334318, 14.21702429, 66.26199243, 32.53524692,
       74.31815025, 44.65728904, 44.77693064, 31.60384474, 53.78295221,
       45.82401099, 53.79045719, 58.11237993, 23.38652868, 34.8301169 ,
       53.83800904, 63.12123903, 53.71423837, 84.55285225, 46.87729385,
       54.61340955, 69.92511044, 71.69812575, 55.87477   , 61.02

In [ ]:
submission

,id,MLM,HLM
0,TEST_000,0,0
1,TEST_001,0,0
2,TEST_002,0,0
3,TEST_003,0,0
4,TEST_004,0,0
...,...,...,...
478,TEST_478,0,0
479,TEST_479,0,0
480,TEST_480,0,0
481,TEST_481,0,0


In [ ]:
# mlm_pred = (cat_pred_1_quantile)*0.5 + ensemble['MLM']*0.5
# hlm_pred = (cat_pred_2_quantile)*0.5 + ensemble['HLM']* 0.5

submission['MLM'] = cat_pred_1_quantile*0.25 + lgbm_pred_1*0.25 + ensemble['MLM']*0.5
submission['HLM'] = cat_pred_2_quantile*0.25 + lgbm_pred_2*0.25 + ensemble['HLM']*0.5

In [ ]:
ensemble = pd.read_csv('ensemble (1).csv')

In [ ]:
ensemble

,id,MLM,HLM
0,TEST_000,24.603051,42.723673
1,TEST_001,61.920015,77.622212
2,TEST_002,37.720648,49.800555
3,TEST_003,43.017860,67.295032
4,TEST_004,56.066593,73.718269
...,...,...,...
478,TEST_478,12.449736,37.194541
479,TEST_479,77.253204,84.552160
480,TEST_480,45.309335,65.568204
481,TEST_481,57.610742,68.744090


In [ ]:
submission.to_csv('ensemble2_cat_lgbm(no_outlier).csv', index=False)
submission

,id,MLM,HLM
0,TEST_000,26.754857,44.727483
1,TEST_001,58.653912,75.483777
2,TEST_002,32.410792,51.768720
3,TEST_003,41.905722,69.575279
4,TEST_004,65.521040,74.920887
...,...,...,...
478,TEST_478,13.310549,35.761358
479,TEST_479,76.980960,86.104547
480,TEST_480,51.500011,67.561634
481,TEST_481,58.405105,72.748207


In [ ]:
submission.to_csv('cat_quantile_lgbm.csv', index=False)

In [ ]:
train_prop.to_csv('train_prop_standard_scale.csv', index=False)
test_prop.to_csv('test_prop_standard_scale.csv', index=False)

In [ ]:
train_prop_quantile.to_csv('train_prop_quantile.csv', index=False)
test_prop_quantile.to_csv('test_prop_quantile.csv', index=False)

In [ ]:
train[['HLM']]

,HLM
0,50.680
1,50.590
2,80.892
3,2.000
4,99.990
...,...
3466,66.126
3467,63.732
3468,93.840
3469,49.519


# Pycaret돌려보기!!!!!!


In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.1 MB/s eta 

In [ ]:
from pycaret.regression import *


In [ ]:
df_prop_mlm = pd.concat([train[['MLM']],train_prop], axis=1)
df_prop_hlm = pd.concat([train[['HLM']],train_prop], axis=1)

In [ ]:
reg = setup(df_prop_hlm, target='HLM')


,Description,Value
0,Session id,4838
1,Target,HLM
2,Target type,Regression
3,Original data shape,"(3471, 1834)"
4,Transformed data shape,"(3471, 1834)"
5,Transformed train set shape,"(2429, 1834)"
6,Transformed test set shape,"(1042, 1834)"
7,Numeric features,1833
8,Preprocess,True
9,Imputation type,simple


In [ ]:
selected_model = create_model('lightgbm',  time_budget=360, error_score='raise')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,29.0612,1198.0101,34.6123,0.0914,1.3291,33.4163
1,26.9515,1031.4180,32.1157,0.2026,1.3321,83.9676
2,27.5405,1056.5242,32.5042,0.2253,1.3160,19.9687
3,27.6908,1115.6936,33.4020,0.1522,1.3784,22.2598
4,25.6456,982.0032,31.3369,0.2106,1.2297,21.3340
5,27.0783,1069.1329,32.6976,0.1149,1.3135,22.0670
6,27.5936,1084.8171,32.9366,0.1936,1.3816,58.4139
7,26.4726,964.1748,31.0512,0.2749,1.2452,14.1240
8,27.5523,1088.5128,32.9926,0.1411,1.3701,46.5970


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tuned_lightgbm_model = tune_model(selected_model, time_budget=360)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,06:55:10
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
import time
start_time = time.time()
# 튜닝 루프
tuned_lightgbm_model = selected_model
while True:
    # 현재 시간 확인
    current_time = time.time()

    # 지정한 시간(초 단위)을 초과하면 종료
    if current_time - start_time > 360:  # 1시간 설정
        break

    # 하이퍼파라미터 튜닝
    tuned_lightgbm_model = tune_model(tuned_lightgbm_model)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,28.8148,1147.7467,33.8784,0.1295,1.3195,42.9222
1,26.8513,1002.1565,31.6569,0.2252,1.3391,89.0918
2,27.6578,1029.1929,32.0810,0.2454,1.3319,24.4578
3,27.6711,1088.3103,32.9895,0.1730,1.4063,25.9066
4,26.4979,986.2415,31.4045,0.2072,1.2374,31.5364
5,26.4101,992.8961,31.5103,0.1780,1.3205,25.3480
6,27.6641,1055.4676,32.4880,0.2154,1.4104,56.2578
7,25.9262,955.4754,30.9108,0.2815,1.2657,14.9565
8,27.6066,1039.4341,32.2403,0.1798,1.3873,48.7160


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
selected_model_xgb = create_model('xgboost')

In [ ]:
# selected_model_cat = tune_model(selected_model_cat)

In [ ]:
# Early stopping 및 최대 시간 설정
custom_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7],
    'min_child_samples': [10, 20, 30],
    'early_stopping_rounds': 10,  # early stopping 적용
    'eval_metric': 'RMSE',
    'time_budget': 360,  # 최대 1시간(초 단위)
}

# LightGBM 모델 하이퍼파라미터 튜닝
tuned_lightgbm_model = tune_model(selected_model, custom_grid=custom_grid)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,06:43:06
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
# RMSE 기준으로 모델들을 정렬
cv_results = cv_results.sort_values(by='RMSE', ascending=False)

# 상위 몇 개의 모델을 선택 (예: 상위 3개 모델 선택)
top_models = cv_results.iloc[:3]['Model'].tolist()

# 선택한 모델들 평가
for model in top_models:
    evaluate_model(model)


In [ ]:
ensemble_models([model_1, model_2, model_3], method='blender')  # 'blender'는 앙상블 방법을 나타냅니다.

In [ ]:
train_prop_quantile

,AlogP_Category,Molecular_Weight_Category,Num_H_Acceptors_Category,Num_H_Donors_Category,Num_RotatableBonds_Category,LogD_Category,Molecular_PolarSurfaceArea_Category,ABC_Category,ABCGG_Category,nAcid_Category,...,SRW10_Category,TSRW10_Category,MW_Category,AMW_Category,WPath_Category,WPol_Category,Zagreb1_Category,Zagreb2_Category,mZagreb1_Category,mZagreb2_Category
0,5.0,7.0,3.0,1.0,5.0,6.0,9.0,6.0,7.0,0.0,...,4.0,7.0,7.0,4.0,7.0,5.0,6.0,5.0,7.0,7.0
1,3.0,2.0,0.0,0.0,0.0,3.0,4.0,2.0,2.0,0.0,...,2.0,3.0,2.0,3.0,1.0,3.0,2.0,2.0,3.0,2.0
2,2.0,2.0,3.0,0.0,1.0,2.0,2.0,3.0,2.0,0.0,...,4.0,4.0,2.0,2.0,2.0,3.0,3.0,3.0,0.0,3.0
3,9.0,9.0,4.0,0.0,3.0,6.0,6.0,9.0,9.0,0.0,...,9.0,9.0,9.0,2.0,9.0,9.0,9.0,9.0,8.0,9.0
4,3.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,8.0,9.0,3.0,0.0,5.0,7.0,3.0,9.0,9.0,0.0,...,9.0,9.0,9.0,5.0,9.0,9.0,9.0,9.0,9.0,9.0
3467,8.0,8.0,4.0,0.0,2.0,8.0,8.0,9.0,9.0,0.0,...,9.0,9.0,8.0,7.0,8.0,9.0,9.0,9.0,8.0,8.0
3468,0.0,6.0,5.0,0.0,2.0,0.0,8.0,7.0,7.0,0.0,...,5.0,7.0,6.0,3.0,7.0,5.0,6.0,6.0,4.0,7.0
3469,0.0,2.0,4.0,0.0,0.0,0.0,8.0,2.0,3.0,0.0,...,1.0,4.0,2.0,6.0,3.0,1.0,2.0,2.0,2.0,3.0


#### -> freq

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')
# test_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')

import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')
# test_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')



train_prop = train_prop.astype(float)
test_prop = test_prop.astype(float)

train_prop.fillna(0, inplace=True)
test_prop.fillna(0, inplace=True)

In [ ]:
train_prop = (train_prop * 10).astype(int)
test_prop = (test_prop * 10).astype(int)

train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,320,40040,500,200,800,320,11730,2130,1740,0,...,1000,7870,40010,760,238000,4000,14200,16500,950,630
1,210,30140,200,100,200,210,7340,1650,1400,0,...,990,6910,30110,750,87000,3500,11200,13200,710,450
2,150,29730,500,0,300,150,6240,1740,1360,200,...,1010,7010,29710,720,102800,3600,12000,14500,520,480
3,470,49460,600,0,500,340,9260,2780,2000,0,...,1060,8610,49420,710,417000,6100,19200,23100,1070,750
4,230,26830,300,0,100,230,4240,1570,1280,0,...,990,5380,26810,740,76200,3200,10600,12500,620,430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,460,34030,400,100,200,470,7170,2110,1580,0,...,1030,7740,34010,800,173000,4300,14800,18000,660,540
3450,420,55340,500,0,700,400,6590,3000,2250,0,...,1050,8810,55210,810,487400,6300,20400,24200,1200,820
3451,430,45950,600,100,400,430,11350,2650,2070,0,...,1060,8420,45910,850,288800,6100,18600,22700,1080,690
3452,-110,38540,700,0,400,-110,10860,2190,1740,0,...,1010,7900,38510,750,226200,4200,14800,17500,800,620


In [ ]:
train_prop['Molecular_Weight'].value_counts()

27830    18
29630    17
30830    16
31130    15
34230    15
         ..
29910     1
19220     1
63570     1
33670     1
52550     1
Name: Molecular_Weight, Length: 1198, dtype: int64

In [ ]:
train_prop['Num_H_Acceptors'].value_counts()

400     984
300     842
500     695
200     381
600     292
700     111
100      91
800      35
900      12
1000      4
1100      3
0         3
1200      1
Name: Num_H_Acceptors, dtype: int64

In [ ]:

# 모든 열에 대해 새로운 칼럼 생성
for col in train_prop.columns:
    value_counts = train_prop[col].value_counts()
    new_col_name = f'freq_{col}'
    train_prop[new_col_name] = train_prop[col].map(value_counts)

print(train_prop)


<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_prop[new_col_name] = train_prop[col].map(value_counts)
<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_prop[new_col_name] = train_prop[col].map(value_counts)
<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

      AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0       320             40040              500           200   
1       210             30140              200           100   
2       150             29730              500             0   
3       470             49460              600             0   
4       230             26830              300             0   
...     ...               ...              ...           ...   
3449    460             34030              400           100   
3450    420             55340              500             0   
3451    430             45950              600           100   
3452   -110             38540              700             0   
3453    560             46650              400             0   

      Num_RotatableBonds  LogD  Molecular_PolarSurfaceArea   ABC  ABCGG  \
0                    800   320                       11730  2130   1740   
1                    200   210                        7340  1650   1400   
2     

<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_prop[new_col_name] = train_prop[col].map(value_counts)
<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_prop[new_col_name] = train_prop[col].map(value_counts)
<ipython-input-351-177971cd81ef>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,freq_SRW10,freq_TSRW10,freq_MW,freq_AMW,freq_WPath,freq_WPol,freq_Zagreb1,freq_Zagreb2,freq_mZagreb1,freq_mZagreb2
0,320,40040,500,200,800,320,11730,2130,1740,0,...,449,12,9,136,2,105,72,42,49,65
1,210,30140,200,100,200,210,7340,1650,1400,0,...,340,20,7,155,3,142,95,50,60,117
2,150,29730,500,0,300,150,6240,1740,1360,200,...,459,18,8,118,4,135,92,39,22,153
3,470,49460,600,0,500,340,9260,2780,2000,0,...,125,5,1,126,1,29,10,5,27,22
4,230,26830,300,0,100,230,4240,1570,1280,0,...,340,6,4,143,3,132,63,40,67,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,460,34030,400,100,200,470,7170,2110,1580,0,...,370,17,18,136,3,101,75,35,55,86
3450,420,55340,500,0,700,400,6590,3000,2250,0,...,210,3,2,126,2,20,14,7,17,11
3451,430,45950,600,100,400,430,11350,2650,2070,0,...,125,8,6,85,1,29,20,11,31,54
3452,-110,38540,700,0,400,-110,10860,2190,1740,0,...,459,12,12,155,2,115,75,21,62,76


In [ ]:
# 모든 열에 대해 새로운 칼럼 생성
for col in test_prop.columns:
    value_counts = test_prop[col].value_counts()
    new_col_name = f'freq_{col}'
    test_prop[new_col_name] = test_prop[col].map(value_counts)

print(test_prop)


<ipython-input-353-b34a402df0cf>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_prop[new_col_name] = test_prop[col].map(value_counts)
<ipython-input-353-b34a402df0cf>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_prop[new_col_name] = test_prop[col].map(value_counts)
<ipython-input-353-b34a402df0cf>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

     AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0      260             36150              400           200   
1       50             37030              500             0   
2      420             34740              400           400   
3      170             34530              500             0   
4      120             35340              400             0   
..     ...               ...              ...           ...   
478    420             30640              200           100   
479    -60             33530              500             0   
480    170             34930              300           100   
481     70             34110              300           200   
482    270             25030              200             0   

     Num_RotatableBonds  LogD  Molecular_PolarSurfaceArea   ABC  ABCGG  nAcid  \
0                   700   260                        9270  1920   1540      0   
1                   300    50                        6830  2120   1680      0   


<ipython-input-353-b34a402df0cf>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_prop[new_col_name] = test_prop[col].map(value_counts)


In [ ]:
train_prop_freq = train_prop.iloc[:,1833:]
test_prop_freq = test_prop.iloc[:,1833:]


In [ ]:
train_prop_freq= train_prop_freq.astype(float)
test_prop_freq = test_prop_freq.astype(float)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_freq, target_1)):
    # 학습데이터
    x_train = train_prop_freq.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_freq.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    # model = CatBoostRegressor(random_state=SEED, verbose=0,cat_features=train_prop_object.columns.tolist())
    model = CatBoostRegressor(random_state=SEED, verbose=0)

    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

0it [00:00, ?it/s]

[31.0593263918636, 33.04051131489214, 32.50419249292038, 30.53586569774758, 31.58985102730241]


31.745949384945227

In [ ]:
model.fit(train_prop_freq, target_1)
cat_pred_1_freq = model.predict(test_prop_freq)

model.fit(train_prop_freq, target_2)
cat_pred_2_freq = model.predict(test_prop_freq)

#### -> 수치로 lgbm***** (2)

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')
# test_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')

import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')
# test_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')



train_prop = train_prop.astype(float)
test_prop = test_prop.astype(float)

train_prop.fillna(0, inplace=True)
test_prop.fillna(0, inplace=True)


In [ ]:
train_prop

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,3.259,400.495,5.0,2.0,8.0,3.259,117.37,21.379612,17.449011,0.0,...,10.081676,78.761075,400.156912,7.695325,2380.0,40.0,142.0,165.0,9.500000,6.361111
1,2.169,301.407,2.0,1.0,2.0,2.172,73.47,16.539255,14.049653,0.0,...,9.907828,69.149596,301.124883,7.528122,870.0,35.0,112.0,132.0,7.138889,4.527778
2,1.593,297.358,5.0,0.0,3.0,1.585,62.45,17.475469,13.660693,2.0,...,10.144510,70.158066,297.170194,7.248054,1028.0,36.0,120.0,145.0,5.277778,4.888889
3,4.771,494.652,6.0,0.0,5.0,3.475,92.60,27.857311,20.034364,0.0,...,10.613467,86.199585,494.246395,7.162991,4170.0,61.0,192.0,231.0,10.784722,7.500000
4,2.335,268.310,3.0,0.0,1.0,2.337,42.43,15.722758,12.817176,0.0,...,9.978363,53.872357,268.121178,7.447810,762.0,32.0,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,4.282,553.480,5.0,0.0,7.0,4.009,65.98,30.031303,22.567906,0.0,...,10.593756,88.164476,552.169496,8.120140,4874.0,63.0,204.0,242.0,12.055556,8.222222
3467,4.304,459.520,6.0,1.0,4.0,4.304,113.53,26.501865,20.702897,0.0,...,10.677177,84.226168,459.136511,8.502528,2888.0,61.0,186.0,227.0,10.895833,6.944444
3468,-1.133,385.417,7.0,0.0,4.0,-1.133,108.65,21.936889,17.403250,0.0,...,10.177818,79.094663,385.175004,7.552451,2262.0,42.0,148.0,175.0,8.000000,6.222222
3469,-0.533,302.289,6.0,1.0,2.0,-0.533,111.44,17.027582,14.772010,0.0,...,9.881191,71.515477,302.112738,8.392021,1133.0,31.0,114.0,134.0,6.777778,4.944444


In [ ]:
from sklearn.preprocessing import StandardScaler

# 샘플 데이터 생성 (2차원 배열 형태로 가정)

# StandardScaler 객체 생성
scaler = StandardScaler()

# 데이터를 표준 스케일링 (평균이 0, 표준편차가 1로 변환)
train_prop = scaler.fit_transform(train_prop)
test_prop = scaler.transform(test_prop)

train_prop = pd.DataFrame(train_prop)
test_prop = pd.DataFrame(test_prop)

train_prop

,0,1,2,3,4,5,6,7,8,9,...,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832
0,0.262846,0.464246,0.654483,0.918903,1.458914,0.348615,1.304380,0.341033,0.473471,-0.235406,...,-0.228016,0.795310,0.490024,-0.301332,0.334827,-0.032840,0.230975,0.141000,0.522473,0.668879
1,-0.416122,-0.784236,-1.331856,-0.104032,-1.085754,-0.304663,-0.215858,-0.742329,-0.677159,-0.235406,...,-0.800300,-0.207337,-0.788709,-0.429319,-0.545618,-0.468238,-0.720353,-0.700746,-0.535412,-0.857409
2,-0.774916,-0.835252,0.654483,-1.126967,-0.661643,-0.657445,-0.597476,-0.532787,-0.808816,5.817155,...,-0.021172,-0.102136,-0.839773,-0.643700,-0.453492,-0.381158,-0.466665,-0.369149,-1.369275,-0.556776
3,1.204680,1.650597,1.316596,-1.126967,0.186580,0.478429,0.446605,1.790863,1.348574,-0.235406,...,1.522576,1.571278,1.704937,-0.708812,1.378533,1.795831,1.816522,1.824493,1.098087,1.617028
4,-0.312720,-1.201248,-0.669743,-1.126967,-1.509866,-0.205500,-1.290760,-0.925076,-1.094333,-0.235406,...,-0.568108,-1.801023,-1.214864,-0.490794,-0.608591,-0.729476,-0.910618,-0.879298,-0.921229,-0.996162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,0.900079,2.391814,0.654483,-1.126967,1.034802,0.799358,-0.475234,2.277443,2.206139,-0.235406,...,1.457688,1.776251,2.452858,0.023847,1.789019,1.969990,2.197053,2.105075,1.667479,2.218293
3467,0.913783,1.207944,1.316596,-0.104032,-0.237532,0.976651,1.171402,1.487489,1.574862,-0.235406,...,1.732302,1.365416,1.251587,0.316550,0.631029,1.795831,1.626257,1.722463,1.147870,1.154517
3468,-2.472959,0.274267,1.978709,-1.126967,-0.237532,-2.290940,1.002410,0.465762,0.457982,-0.235406,...,0.088474,0.830109,0.296573,-0.410696,0.266024,0.141319,0.421241,0.396075,-0.149595,0.553252
3469,-2.099215,-0.773123,1.316596,-0.104032,-1.085754,-1.930345,1.099026,-0.633032,-0.432652,-0.235406,...,-0.887987,0.039466,-0.775954,0.231961,-0.392269,-0.816556,-0.656931,-0.649731,-0.697206,-0.510525


In [ ]:
from lightgbm import LGBMRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop, target_1)):
    # 학습데이터
    x_train = train_prop.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = LGBMRegressor(random_state=SEED, objective='regression')
    model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric="rmse")
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    # HLM
    model = LGBMRegressor(random_state=SEED, objective='regression')
    model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric="rmse")
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)


    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

0it [00:00, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075971 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 283242
[LightGBM] [Info] Number of data points in the train set: 2776, number of used features: 1327
[LightGBM] [Info] Start training from score 37.395107
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.083138 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 283242
[LightGBM] [Info] Number of data points in the train set: 2776, number of used features: 1327
[LightGBM] [Info] Start training from score 53.148670
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 283359
[LightGBM] [Info] Number of data points in the train set: 2777, number of used features: 1327
[LightG

31.4147973281615

In [ ]:
model.fit(train_prop, target_1)
lgbm_pred_1 = model.predict(test_prop)

model.fit(train_prop, target_2)
lgbm_pred_2 = model.predict(test_prop)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062010 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 285372
[LightGBM] [Info] Number of data points in the train set: 3471, number of used features: 1331
[LightGBM] [Info] Start training from score 37.391365
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.064015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 285372
[LightGBM] [Info] Number of data points in the train set: 3471, number of used features: 1331
[LightGBM] [Info] Start training from score 53.179643


In [ ]:
lgbm_pred_1

array([38.70987429, 58.55054045, 26.48444773, 40.91705026, 81.24352741,
       44.90833308, 30.27195551, 18.83121035, 25.42886025, 13.82888882,
       39.85872072, 44.91450694, 56.81619172, 33.23264961, 27.45902642,
       58.53043738, 38.90601785, 51.95209867, 71.89582211, 48.73347179,
       20.19667369,  4.07045645,  7.0456677 , 17.60253955, 20.69172533,
       21.69538615, 28.04115098, 61.43620402, 29.35706897, 29.9022493 ,
       59.99217777, 22.84368861, 22.91445631, 38.82493222,  9.11010912,
       20.47959005, 41.01155282, 55.93299021, 22.43140886, 30.0667581 ,
       39.48007362, 66.56851888, 10.71898972,  7.79995754,  8.03736783,
       61.34020116,  4.44485192, 10.77771452, 57.95129599, 30.89357009,
       67.9506264 , 10.49420351, 33.51123851, 34.2483122 , 26.48811896,
       11.74424446, 43.15686325, 27.73597516, 18.70899077, 54.93851771,
       22.52247332, 62.98775698, 51.85335238, 66.38649889, 24.39710677,
       39.80434079, 45.92698141, 57.79370397, 51.64072546, 39.79

#### -> 표준편차 구하기

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

target_1 = train['MLM']
target_2 = train['HLM']

In [ ]:
# train_prop = train.iloc[:, 4:]
# test_prop = test.iloc[:, 2:]

# import pandas as pd

# # train_prop_object의 모든 열을 처리
# for column_name in train_prop.columns:
#     train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')

# # test_prop_object의 모든 열을 처리
# for column_name in train_prop.columns:
#     train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')


# import pandas as pd

# # train_prop_object의 모든 열을 처리
# for column_name in test_prop.columns:
#     test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')

# # test_prop_object의 모든 열을 처리
# for column_name in test_prop.columns:
#     test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')




# train_prop = train_prop.astype(float)
# test_prop = test_prop.astype(float)

# train_prop.fillna(0, inplace=True)
# test_prop.fillna(0, inplace=True)


In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in train_prop.columns:
    train_prop[column_name] = pd.to_numeric(train_prop[column_name], errors='coerce')




In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in test_prop.columns:
    test_prop[column_name] = pd.to_numeric(test_prop[column_name], errors='coerce')




In [ ]:
train_prop = train_prop.astype(float)
test_prop = test_prop.astype(float)

In [ ]:
train_prop.fillna(0, inplace=True)
test_prop.fillna(0, inplace=True)

In [ ]:
train_prop.isna().sum().sum(), test_prop.isna().sum().sum()

(0, 0)

In [ ]:
std_deviation = train_prop['AATS0Z'].std()

# 결과 출력
print(std_deviation)

14.295289503374


In [ ]:
# print(train_prop)

import pandas as pd

# 각 열을 0~10 범주로 나누기
num_bins = 10  # 원하는 범주 수

for col in train_prop.columns:
    bin_edges = pd.qcut(train_prop[col], q=num_bins, labels=False, duplicates='drop')
    bin_labels = [str(i) for i in range(num_bins)]
    train_prop[f'{col}_Category'] = pd.Categorical(bin_edges, categories=range(num_bins), ordered=True)
    train_prop[f'{col}_Category'] = train_prop[f'{col}_Category'].cat.rename_categories(bin_labels)

# 결과 출력
print(train_prop)


#### -> CatBoost 범주화

In [ ]:
import numpy as np

# 데이터
data = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55]

# 데이터의 최소값과 최대값 계산
min_value = min(data)
max_value = max(data)

# 원하는 구간(간격) 크기 설정
interval_size = 10  # 예시로 10 단위로 구간화

# 구간(간격)화하여 각 데이터에 대한 범주값 부여
categories = [(x - min_value) // interval_size for x in data]

# 결과 출력
print(categories)


In [ ]:
cat_pred_1_quantile

array([17.72612877, 53.02278879, 26.45941938, 43.63081152, 65.67503524,
       42.93265486, 21.39982559, 28.46380056, 28.1423221 , 17.12020791,
       39.99701064, 52.61660192, 54.20386369, 26.98268465, 17.13696642,
       65.65345216, 37.89579124, 44.60636452, 60.00585839, 47.81215687,
       26.59092859,  6.65711067, 10.32914822, 27.029497  , 22.6426644 ,
       23.33997535, 29.63225129, 67.46214733, 21.63510211, 44.76053791,
       61.07546938, 25.97658689, 21.50585439, 31.83651647, 21.17127191,
       27.65694068, 39.74024685, 51.18540663,  9.52018313, 22.07206192,
       28.92489472, 66.53812098, 25.08856251, 15.02233297,  7.24589059,
       58.2855841 ,  5.14337549,  1.1210477 , 53.86548772, 16.5790769 ,
       68.06059715, 23.92271954, 31.74190814, 25.4179383 , 22.57969527,
       17.71347555, 35.45802428, 37.20069022,  8.64205997, 46.80685663,
       24.92651726, 46.83377933, 32.22224369, 67.73890408, 20.3461009 ,
       37.70599425, 45.39690963, 63.7336241 , 23.94056812, 39.90

In [ ]:
ensemble = pd.read_csv(f'ensemble (2).csv')
ensemble

,id,MLM,HLM
0,TEST_000,24.603051,42.723673
1,TEST_001,61.920015,77.622212
2,TEST_002,37.720648,49.800555
3,TEST_003,43.017860,67.295032
4,TEST_004,56.066593,73.718269
...,...,...,...
478,TEST_478,12.449736,37.194541
479,TEST_479,77.253204,84.552160
480,TEST_480,45.309335,65.568204
481,TEST_481,57.610742,68.744090


In [ ]:
mlm_pred = (cat_pred_1_quantile)*0.5 + ensemble['MLM']*0.5
hlm_pred = (cat_pred_2_quantile)*0.5 + ensemble['HLM']* 0.5

In [ ]:
submission['MLM'] = cat_pred_1_quantile
submission['HLM'] = cat_pred_2_quantile

In [ ]:
submission

,id,MLM,HLM
0,TEST_000,17.726129,42.447446
1,TEST_001,53.022789,75.562687
2,TEST_002,26.459419,49.493749
3,TEST_003,43.630812,71.597876
4,TEST_004,65.675035,67.738244
...,...,...,...
478,TEST_478,19.777566,40.794860
479,TEST_479,71.450739,83.240859
480,TEST_480,48.869908,62.576153
481,TEST_481,67.433279,79.461378


In [ ]:
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
submission.to_csv("submission_ML23(cat_quantile).csv", index=False)

# CatBoost - SMILES
- https://catboost.ai/en/docs/concepts/python-reference_tokenizer

In [ ]:
# class Tokenizer(lowercasing=None,
#                 lemmatizing=None,
#                 number_process_policy=None,
#                 number_token=None,
#                 separator_type=None,
#                 delimiter=None,
#                 split_by_set=None,
#                 skip_empty=None,
#                 token_types=None,
#                 sub_tokens_policy=None,
#                 languages=None)

In [ ]:
from catboost.text_processing import Tokenizer


text="Still, I would love to see you at 12, if you don't mind"

tokenized = Tokenizer(lowercasing=True,
                      separator_type='BySense',
                      token_types=['Word', 'Number']).tokenize(text)

print (tokenized)


['still', 'i', 'would', 'love', 'to', 'see', 'you', 'at', '12', 'if', 'you', "don't", 'mind']


In [ ]:
text = train['SMILES'][0]

result = Tokenizer(lowercasing=False, separator_type='BySense', token_types=['Word', 'Number']).tokenize(text)

print(result)

['CNC', 'O', 'C', 'n', 'nH', '2']


In [ ]:
cat_token = []

for text in train['SMILES']:
    result = Tokenizer(lowercasing=False, separator_type='BySense').tokenize(text)
    print(text, ':' , result)
    cat_token.append(result)

CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC : ['CCOc1ccc', 'CNC', 'O', 'c2cc', 'c3sc', 'C', 'nc3C', 'n', 'nH', '2', 'cc1OCC']
Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1 : ['Cc1nc', 'C', 'c', 'CN2CC', 'C', 'C', 'O', 'Nc3ccccc32', 's1']
CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1 : ['CCCN1CCN', 'c2nn3nnnc3c3ccccc23', 'CC1']
Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC5)CC4)c3C)nn2)cc1 : ['Cc1ccc', 'c2ccc', 'n3nc', 'C', 'c', 'S', 'O', 'O', 'N4CCN', 'C5CCCCC5', 'CC4', 'c3C', 'nn2', 'cc1']
Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2 : ['Cc1ccc2c', 'c1', 'N', 'C', 'O', 'c1ccncc1', 'CC', 'C', 'O2']
COc1c(F)c(F)cc2c(=O)c(C(=O)NCCCN3CCCC3=O)cn(C3CC3)c12 : ['COc1c', 'F', 'c', 'F', 'cc2c', 'O', 'c', 'C', 'O', 'NCCCN3CCCC3', 'O', 'cn', 'C3CC3', 'c12']
CN(C)c1cccc2c(S(=O)(=O)NC(CC(=O)O)C(=O)O)cccc12 : ['CN', 'C', 'c1cccc2c', 'S', 'O', 'O', 'NC', 'CC', 'O', 'O', 'C', 'O', 'O', 'cccc12']
C1=Cn2nc(/C=C/c3cccs3)nc2-c2ccccc2O1 : ['C1', 'Cn2nc', 'C', 'C', 'c3cccs3', 'nc2', 'c2ccccc2O1']
COc1ccc2nc(C3=CCCN(C)C3)n(C)c2c1 : [

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 여러 개의 텍스트 데이터가 리스트로 구성된 경우
text_data = [
    'Cc1ccccc1',
    'c1nc2ccccc2cc1C',
    'C',
    'n1c',
    'O',
    'nH',
    'c2c',
    'S',
    'C',
    'O',
    'O',
    'ncnc21'
]

# TfidfVectorizer 객체 생성
tfidf_vectorizer = TfidfVectorizer()

# 텍스트 데이터를 TF-IDF로 변환
X_tfidf = tfidf_vectorizer.fit_transform(text_data)

# TF-IDF로 변환된 데이터를 밀집 행렬(Dense Matrix)로 변환
X_tfidf_dense = X_tfidf.toarray()

# TF-IDF로 변환된 데이터 출력
print(X_tfidf_dense)

# 특성(단어) 목록 출력
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()
print("Feature Names:", feature_names_tfidf)


[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
Feature Names: ['c1nc2ccccc2cc1c' 'c2c' 'cc1ccccc1' 'n1c' 'ncnc21' 'nh']


In [ ]:
cat_token

[['CCOc1ccc',
  'CNC',
  'O',
  'c2cc',
  'c3sc',
  'C',
  'nc3C',
  'n',
  'nH',
  '2',
  'cc1OCC'],
 ['Cc1nc', 'C', 'c', 'CN2CC', 'C', 'C', 'O', 'Nc3ccccc32', 's1'],
 ['CCCN1CCN', 'c2nn3nnnc3c3ccccc23', 'CC1'],
 ['Cc1ccc',
  'c2ccc',
  'n3nc',
  'C',
  'c',
  'S',
  'O',
  'O',
  'N4CCN',
  'C5CCCCC5',
  'CC4',
  'c3C',
  'nn2',
  'cc1'],
 ['Cc1ccc2c', 'c1', 'N', 'C', 'O', 'c1ccncc1', 'CC', 'C', 'O2'],
 ['COc1c',
  'F',
  'c',
  'F',
  'cc2c',
  'O',
  'c',
  'C',
  'O',
  'NCCCN3CCCC3',
  'O',
  'cn',
  'C3CC3',
  'c12'],
 ['CN',
  'C',
  'c1cccc2c',
  'S',
  'O',
  'O',
  'NC',
  'CC',
  'O',
  'O',
  'C',
  'O',
  'O',
  'cccc12'],
 ['C1', 'Cn2nc', 'C', 'C', 'c3cccs3', 'nc2', 'c2ccccc2O1'],
 ['COc1ccc2nc', 'C3', 'CCCN', 'C', 'C3', 'n', 'C', 'c2c1'],
 ['CCN1CCCC1CNC', 'O', 'C1CCCN', 'c2ncnc3', 'nH', 'cnc23', 'C1'],
 ['CC1CN', 'Cc2cncs2', 'CCN1CC', 'C', 'C', 'O'],
 ['Nc1nc', 'N2CCN', 'c3ccccc3Cl', 'CC2', 'nc2ccccc12'],
 ['CC', 'C', 'c1ccc', 'CCCNS', 'C', 'O', 'O', 'cc1'],
 ['Oc1cc',

In [ ]:
train_prop_smiles = train[['id','SMILES']]
test_prop_smiles = test[['id','SMILES']]

train_prop_smiles

,id,SMILES
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2
...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1


#### 학습 및 검증

In [ ]:
!pip install CatBoost

In [ ]:
# model = CatBoostRegressor(iterations=100, depth=6, cat_features=['Category'])
# model.fit(data[['Category', 'Value1', 'Value2']], data['Target'])  # Label을 Target으로 변경

In [ ]:
from catboost import CatBoostClassifier


In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_smiles, target_1)):
    # 학습데이터
    x_train = train_prop_smiles.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_smiles.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = CatBoostRegressor(random_state=SEED, verbose=0,cat_features=['id','SMILES'])
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0, cat_features=['id','SMILES'])
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

0it [00:00, ?it/s]

[35.208931671707, 35.85626709565258, 36.27827306631629, 35.708996431020964, 35.99149038328164]


35.8087917295957

In [ ]:
model.fit(train_prop_smiles, target_1)
cat_pred_1 = model.predict(test_prop_smiles)

model.fit(train_prop_smiles, target_2)
cat_pred_2 = model.predict(test_prop_smiles)

In [ ]:
from rdkit import Chem
import numpy as np

# 토큰화된 SMILES 데이터
smiles_list = train['SMILES'].tolist()

# 화학 분자 객체로 변환하고 수치 임베딩 추출
mol_objects = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# 간단한 화학적 특성 추출 예시 (다양한 특성을 추출하려면 더 많은 작업 필요)
mol_features = [Chem.Descriptors.MolWt(mol) for mol in mol_objects]

# 수치 임베딩을 NumPy 배열로 변환
embedding = np.array(mol_features)

# 임베딩된 데이터 출력
print("Embedding for SMILES:")
for smiles, embed in zip(smiles_list, embedding):
    print(f"{smiles}: {embed}")


Embedding for SMILES:
CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC: 400.50400000000013
Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1: 301.415
CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1: 297.3660000000001
Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC5)CC4)c3C)nn2)cc1: 494.6650000000004
Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2: 268.31600000000003
COc1c(F)c(F)cc2c(=O)c(C(=O)NCCCN3CCCC3=O)cn(C3CC3)c12: 419.4280000000002
CN(C)c1cccc2c(S(=O)(=O)NC(CC(=O)O)C(=O)O)cccc12: 366.39500000000004
C1=Cn2nc(/C=C/c3cccs3)nc2-c2ccccc2O1: 293.35100000000006
COc1ccc2nc(C3=CCCN(C)C3)n(C)c2c1: 257.33699999999993
CCN1CCCC1CNC(=O)C1CCCN(c2ncnc3[nH]cnc23)C1: 357.46200000000005
CC1CN(Cc2cncs2)CCN1CC(C)(C)O: 269.41399999999993
Nc1nc(N2CCN(c3ccccc3Cl)CC2)nc2ccccc12: 339.83000000000004
CC(C)c1ccc(CCCNS(C)(=O)=O)cc1: 255.38299999999998
Oc1cc(-c2ccc(-c3ccc(Br)cc3)cc2)nn1-c1ccccn1: 392.25600000000003
COc1cc(Nc2ncc(-c3cnn(C)c3)c(Nc3ccc4c(c3)CNCC4)n2)cc(OC)c1: 457.5380000000003
Cc1[nH]c(=O)[nH]c(=O)c1S(=O)(=O)NCC1CCCO1: 289.31300000000005

# CatBoost - 모든 칼럼들을 범주화하기

In [ ]:
train_prop

# RDKit API

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# 예제 SMILES 문자열
smiles_string = "C1CCCC1"

# SMILES 문자열을 RDKit Mol 객체로 변환
mol = Chem.MolFromSmiles(smiles_string)

# Mol 객체가 정상적으로 생성되었는지 확인
if mol is not None:
    # 특징 추출 예제: 분자량(Molecular Weight) 계산
    molecular_weight = Descriptors.MolWt(mol)

    print("SMILES 문자열:", smiles_string)
    print("분자량(Molecular Weight):", molecular_weight)
else:
    print("SMILES 문자열을 Mol 객체로 변환할 수 없습니다.")


SMILES 문자열: C1CCCC1
분자량(Molecular Weight): 70.135


# 학습, 검증, 추론

## 데이터 분리 ***

- 1) pure object columns 삭제

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


In [ ]:
train_prop.drop(columns = pure_object_cols, inplace=True)
test_prop.drop(columns = pure_object_cols, inplace=True)

In [ ]:
train_prop.shape, test_prop.shape

((3454, 1514), (483, 1514))

In [ ]:
# cols = ['mol','mode_label_proba']

# train = train.drop(columns = cols,axis=1)
# test = test.drop(columns = cols,axis=1)

- 2) float object columns

In [ ]:
train_prop_object = train_prop[float_object_cols]
test_prop_object = test_prop[float_object_cols]


- 3) SMILES, ID

In [ ]:
train.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111


In [ ]:
train_prop_smiles = train[['id','SMILES']]
test_prop_smiles = test[['id','SMILES']]

- 4) float columns

In [ ]:
train_prop_float = train_prop[float_columns]
test_prop_float = test_prop[float_columns]

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [ ]:
from sklearn.metrics import make_scorer

rmse_score = make_scorer(rmse, greater_is_better=False)

### A. object(float) columns

In [ ]:
train_prop_object.isna().sum().sum(), test_prop_object.isna().sum().sum()

(0, 0)

#### CatBoost

In [ ]:
!pip install CatBoost

In [ ]:
from catboost import CatBoostClassifier


In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop_object.columns:
    train_prop_object[column_name] = pd.to_numeric(train_prop_object[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in test_prop_object.columns:
    test_prop_object[column_name] = pd.to_numeric(test_prop_object[column_name], errors='coerce')



In [ ]:
train_prop_object.isna().sum().sum(), test_prop_object.isna().sum().sum()

(196428, 27480)

In [ ]:
train_prop_object.shape[0] * train_prop_object.shape[1]



880770

In [ ]:
train_prop_object.fillna(0, inplace=True)
test_prop_object.fillna(0, inplace=True)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_object, target_1)):
    # 학습데이터
    x_train = train_prop_object.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_object.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [12:03, 144.76s/it]

[31.845847257436787, 32.568143462806326, 33.018936493098664, 31.322089283807486, 32.120797577145446]


32.17516281485894

In [ ]:
model.fit(train_prop_object, target_1)
cat_pred_1 = model.predict(test_prop_object)

model.fit(train_prop_object, target_2)
cat_pred_2 = model.predict(test_prop_object)

### B. float columns

In [ ]:
train_prop_float.isna().sum().sum(), test_prop_float.isna().sum().sum()

(2, 1)

In [ ]:
# 결측치 채우기
train_prop_float["AlogP"] = np.where(pd.isna(train_prop_float["AlogP"]), train_prop_float["LogD"], train_prop_float["AlogP"])
test_prop_float["AlogP"] = np.where(pd.isna(test_prop_float["AlogP"]), test_prop_float["LogD"], test_prop_float["AlogP"])

- 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일링을 위한 객체 생성
scaler = MinMaxScaler()

# Train 데이터를 스케일링하고 변환
train_prop_float = scaler.fit_transform(train_prop_float)

# Test 데이터를 스케일링하고 변환
test_prop_float = scaler.transform(test_prop_float)


In [ ]:
train_prop_float = pd.DataFrame(train_prop_float)
train_prop_float

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.488883,0.408759,0.416667,0.250,0.470588,0.525975,0.546835,0.405597,0.417590,0.000000,...,0.544667,0.579584,0.408596,0.137182,0.214867,0.305263,0.373016,0.345794,0.345075,0.466563
1,0.394950,0.231359,0.166667,0.125,0.117647,0.439098,0.336496,0.264882,0.279238,0.000000,...,0.485644,0.436791,0.231203,0.127029,0.072064,0.252632,0.253968,0.242991,0.222861,0.261275
2,0.345312,0.224110,0.416667,0.000,0.176471,0.392184,0.283695,0.292099,0.263407,0.666667,...,0.566000,0.451774,0.224119,0.110023,0.087006,0.263158,0.285714,0.283489,0.126528,0.301711
3,0.619183,0.577331,0.500000,0.000,0.294118,0.543238,0.428154,0.593912,0.522812,0.000000,...,0.725217,0.690095,0.577135,0.104858,0.384150,0.526316,0.571429,0.551402,0.411574,0.594090
4,0.409255,0.172104,0.250000,0.000,0.058824,0.452286,0.187773,0.241146,0.229077,0.000000,...,0.509591,0.209825,0.172085,0.122153,0.061850,0.221053,0.230159,0.221184,0.178289,0.242613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,0.605222,0.301135,0.333333,0.125,0.117647,0.644022,0.328255,0.400066,0.353467,0.000000,...,0.633958,0.559821,0.301096,0.161673,0.153395,0.336842,0.396825,0.392523,0.195543,0.370140
3450,0.577042,0.682652,0.416667,0.000,0.411765,0.585918,0.300608,0.657112,0.625926,0.000000,...,0.718524,0.719286,0.680891,0.162977,0.450728,0.547368,0.619048,0.585670,0.477354,0.674961
3451,0.578938,0.514433,0.500000,0.125,0.235294,0.609495,0.528437,0.554507,0.550021,0.000000,...,0.746847,0.660777,0.514244,0.186196,0.262909,0.526316,0.547619,0.538941,0.417326,0.531882
3452,0.110393,0.381764,0.583333,0.000,0.235294,0.174952,0.505055,0.421798,0.415727,0.000000,...,0.577309,0.584540,0.381759,0.128507,0.203707,0.326316,0.396825,0.376947,0.267434,0.451011


In [ ]:
test_prop_float = pd.DataFrame(test_prop_float)
test_prop_float

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.435626,0.338954,0.333333,0.250,0.411765,0.476103,0.428921,0.343099,0.335359,0.0,...,0.495111,0.485416,0.338802,0.091685,0.141763,0.242105,0.309524,0.280374,0.242991,0.385692
1,0.258445,0.354877,0.416667,0.000,0.176471,0.312260,0.311772,0.401242,0.394503,0.0,...,0.629119,0.540223,0.354851,0.128609,0.165027,0.357895,0.388889,0.376947,0.259166,0.423017
2,0.576525,0.313726,0.333333,0.500,0.294118,0.608376,0.429400,0.370016,0.337512,0.0,...,0.519708,0.309983,0.313691,0.118442,0.169945,0.273684,0.333333,0.299065,0.287563,0.395023
3,0.362720,0.310045,0.416667,0.000,0.117647,0.408967,0.373581,0.396886,0.361725,0.0,...,0.614867,0.560326,0.310035,0.181057,0.148005,0.326316,0.388889,0.383178,0.163911,0.379471
4,0.313082,0.324475,0.333333,0.000,0.117647,0.279012,0.277466,0.383519,0.361273,0.0,...,0.646057,0.536702,0.324457,0.107583,0.146397,0.378947,0.380952,0.380062,0.304817,0.385692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0.570579,0.240375,0.166667,0.125,0.411765,0.601742,0.248622,0.257693,0.306877,0.0,...,0.470257,0.444321,0.240349,0.027504,0.097409,0.221053,0.238095,0.221184,0.254853,0.339036
479,0.155636,0.292214,0.416667,0.000,0.058824,0.126758,0.320636,0.329945,0.351373,0.0,...,0.615006,0.503620,0.292213,0.085277,0.110838,0.326316,0.325397,0.320872,0.329619,0.334370
480,0.362461,0.317251,0.250000,0.125,0.176471,0.408728,0.318528,0.384347,0.390170,0.0,...,0.668779,0.542689,0.317216,0.141032,0.128712,0.389474,0.380952,0.383178,0.304817,0.382582
481,0.276112,0.302479,0.250000,0.250,0.117647,0.299313,0.318145,0.230130,0.246490,0.0,...,0.474406,0.205264,0.300814,0.358055,0.067902,0.210526,0.214286,0.199377,0.287563,0.242613


In [ ]:
# import joblib

# # 모델 저장
# joblib.dump(mlm_model, 'mlm_model.pkl')
# joblib.dump(hlm_model, 'hlm_model.pkl')

#### AdaBoost

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop_float, target_1)):
#     # 학습데이터
#     x_train = train_prop_float.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop_float.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = AdaBoostRegressor(random_state=SEED)
#     model.fit(x_train, y_train_1)
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     #HLM
#     model = AdaBoostRegressor(random_state=SEED)
#     model.fit(x_train, y_train_2)
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)

#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop_float, target_1)
# ada_pred_1 = model.predict(test_prop_float)

# model.fit(train_prop_float, target_2)
# ada_pred_2 = model.predict(test_prop_float)

#### randomforest

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_float, target_1)):
    # 학습데이터
    x_train = train_prop_float.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_float.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [46:39, 559.97s/it]

[30.978366949192015, 32.575945789114286, 31.787306579152258, 30.688974175121523, 31.32539794849395]


31.47119828821481

In [ ]:
model.fit(train_prop_float, target_1)
rf_pred_1 = model.predict(test_prop_float)

model.fit(train_prop_float, target_2)
rf_pred_2 = model.predict(test_prop_float)

#### XGBRegressor

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop, target_1)):
#     # 학습데이터
#     x_train = train_prop.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = XGBRegressor(random_state=SEED)
#     model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric='rmse')
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     # HLM
#     model = XGBRegressor(random_state=SEED)
#     model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric='rmse')
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)

#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop, target_1)
# xgb_pred_1 = model.predict(test_prop)

# model.fit(train_prop, target_2)
# xgb_pred_2 = model.predict(test_prop)

#### LGBMRegressor

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop, target_1)):
#     # 학습데이터
#     x_train = train_prop.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = LGBMRegressor(random_state=SEED, objective='regression')
#     model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric="rmse")
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     # HLM
#     model = LGBMRegressor(random_state=SEED, objective='regression')
#     model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric="rmse")
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)


#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop, target_1)
# lgbm_pred_1 = model.predict(test_prop)

# model.fit(train_prop, target_2)
# lgbm_pred_2 = model.predict(test_prop)

In [ ]:
# mlm_pred = (cat_pred_1 + ada_pred_1 + rf_pred_1 + xgb_pred_1 + lgbm_pred_1) / 5
# hlm_pred = (cat_pred_2 + ada_pred_2 +rf_pred_2 + xgb_pred_2 + lgbm_pred_2) / 5

In [ ]:
mlm_pred = (cat_pred_1)*0.5 + (rf_pred_1)*0.5
hlm_pred = (cat_pred_2)*0.5 + (rf_pred_2)* 0.5

In [ ]:
submission['MLM'] = mlm_pred
submission['HLM'] = hlm_pred

In [ ]:
submission

,id,MLM,HLM
0,TEST_000,29.346688,47.805495
1,TEST_001,49.700280,74.044434
2,TEST_002,39.409920,54.119813
3,TEST_003,45.239855,67.436936
4,TEST_004,50.288453,70.644674
...,...,...,...
478,TEST_478,26.633371,47.933798
479,TEST_479,61.370739,75.454900
480,TEST_480,44.359946,64.263805
481,TEST_481,53.342733,68.428288


In [ ]:
submission.to_csv("submission_ML20(cat).csv", index=False)

In [ ]:
submission

In [ ]:
sub_16 = pd.read_csv('submission_ML16(preprocess).csv')

In [ ]:
sub_16

In [ ]:
sub_10 = pd.read_csv('submission_ML10(preprocess).csv')

In [ ]:
sub_10